In [1]:
import os, re
from spacy.lang.id import Indonesian
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [3]:
#TERM WEIGHTING

def termFrequencyInDoc(vocab, doc_dict):
    tf_docs = {}
    for doc_id in doc_dict.keys():
        tf_docs[doc_id] = {}
    for word in vocab:
        for doc_id,doc in doc_dict.items():
            tf_docs[doc_id][word] = doc.count(word)
    return tf_docs

In [4]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
def stemming(text):
    # create stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    # stemming process
    output   = stemmer.stem(text)
    return output

In [5]:
#INVERTED INDEX
# mendapatkan inverted index pada corpus di atas dengan menambahkan fungsi stemming agar menjadi kata dasar
inverted_index = {}

doc1_term = ["pengembangan", "sistem", "informasi", "penjadwalan", "mata", "kuliah", "information", "retrieval"]
doc2_term = ["pengembangan", "model", "analisis", "sentimen",  "berita"]
doc3_term = ["pengembangan", "analisis", "sistem", "input", "output", "dalam", "information", "retrieval"]

corpus_term = [doc1_term, doc2_term, doc3_term]

 
for i in range(len(corpus_term)):
    for item in corpus_term[i]: 
        item = stemming(item) #funsgi stemming
        if item not in inverted_index:
            inverted_index[item] = []
        if (item in inverted_index) and ((i+1) not in inverted_index[item]):
            inverted_index[item].append(i+1)

vocab = list(inverted_index.keys())
doc_dict = {}

#clean after stemming

doc_dict['doc1'] = "kembang sistem informasi jadwal"
doc_dict['doc2'] = "kembang model analisis sentimen berita"
doc_dict['doc3'] = "analisis sistem input output"

print(termFrequencyInDoc(vocab, doc_dict))

{'doc1': {'kembang': 1, 'sistem': 1, 'informasi': 1, 'jadwal': 1, 'mata': 0, 'kuliah': 0, 'information': 0, 'retrieval': 0, 'model': 0, 'analisis': 0, 'sentimen': 0, 'berita': 0, 'input': 0, 'output': 0, 'dalam': 0}, 'doc2': {'kembang': 1, 'sistem': 0, 'informasi': 0, 'jadwal': 0, 'mata': 0, 'kuliah': 0, 'information': 0, 'retrieval': 0, 'model': 1, 'analisis': 1, 'sentimen': 1, 'berita': 1, 'input': 0, 'output': 0, 'dalam': 0}, 'doc3': {'kembang': 0, 'sistem': 1, 'informasi': 0, 'jadwal': 0, 'mata': 0, 'kuliah': 0, 'information': 0, 'retrieval': 0, 'model': 0, 'analisis': 1, 'sentimen': 0, 'berita': 0, 'input': 1, 'output': 1, 'dalam': 0}}


In [6]:
def tokenisasi(text):
    tokens = text.split(" ")
    return tokens

In [7]:
# Menghitung dokumen frequency dan inverse document frequency

def wordDocFre(vocab, doc_dict):
    df = {}
    for word in vocab:
        frq = 0
        for doc in doc_dict.values():
            if word in tokenisasi(doc):
                frq = frq + 1
        df[word] = frq
    return df
print(wordDocFre(vocab, doc_dict))
    

{'kembang': 2, 'sistem': 2, 'informasi': 1, 'jadwal': 1, 'mata': 0, 'kuliah': 0, 'information': 0, 'retrieval': 0, 'model': 1, 'analisis': 2, 'sentimen': 1, 'berita': 1, 'input': 1, 'output': 1, 'dalam': 0}


In [8]:
import numpy as np

def inverseDocFre(vocab, doc_fre,length):
    idf = {}
    for word in vocab:
        idf[word] = idf[word] = 1+ np.log((length + 1)/(doc_fre[word]+1))
    return idf

In [9]:
print(inverseDocFre(vocab,wordDocFre(vocab, doc_dict),len(doc_dict)))

{'kembang': 1.2876820724517808, 'sistem': 1.2876820724517808, 'informasi': 1.6931471805599454, 'jadwal': 1.6931471805599454, 'mata': 2.386294361119891, 'kuliah': 2.386294361119891, 'information': 2.386294361119891, 'retrieval': 2.386294361119891, 'model': 1.6931471805599454, 'analisis': 1.2876820724517808, 'sentimen': 1.6931471805599454, 'berita': 1.6931471805599454, 'input': 1.6931471805599454, 'output': 1.6931471805599454, 'dalam': 2.386294361119891}


In [10]:
#VECTOR SPACE MODEL

def tfidf(vocab, tf, idf_scr, doc_dict):
    tf_idf_scr = {}
    for doc_id in doc_dict.keys():
        tf_idf_scr[doc_id] = {}
    for word in vocab:
        for doc_id,doc in doc_dict.items():
            tf_idf_scr[doc_id][word] = tf[doc_id][word] * idf_scr[word]
    return tf_idf_scr


In [11]:
tf_idf = tfidf(vocab, termFrequencyInDoc(vocab, doc_dict), inverseDocFre(vocab, wordDocFre(vocab, doc_dict),len(doc_dict)), doc_dict)
print(tf_idf)

# Term - Document, Matrix

TD = np.zeros((len(vocab),len(doc_dict)))
for word in vocab:
    for doc_id, doc in tf_idf.items():
        ind1 = vocab.index(word)
        ind2 = list(tf_idf.keys()).index(doc_id)
        TD[ind1][ind2] = tf_idf[doc_id][word]
print(TD)

{'doc1': {'kembang': 1.2876820724517808, 'sistem': 1.2876820724517808, 'informasi': 1.6931471805599454, 'jadwal': 1.6931471805599454, 'mata': 0.0, 'kuliah': 0.0, 'information': 0.0, 'retrieval': 0.0, 'model': 0.0, 'analisis': 0.0, 'sentimen': 0.0, 'berita': 0.0, 'input': 0.0, 'output': 0.0, 'dalam': 0.0}, 'doc2': {'kembang': 1.2876820724517808, 'sistem': 0.0, 'informasi': 0.0, 'jadwal': 0.0, 'mata': 0.0, 'kuliah': 0.0, 'information': 0.0, 'retrieval': 0.0, 'model': 1.6931471805599454, 'analisis': 1.2876820724517808, 'sentimen': 1.6931471805599454, 'berita': 1.6931471805599454, 'input': 0.0, 'output': 0.0, 'dalam': 0.0}, 'doc3': {'kembang': 0.0, 'sistem': 1.2876820724517808, 'informasi': 0.0, 'jadwal': 0.0, 'mata': 0.0, 'kuliah': 0.0, 'information': 0.0, 'retrieval': 0.0, 'model': 0.0, 'analisis': 1.2876820724517808, 'sentimen': 0.0, 'berita': 0.0, 'input': 1.6931471805599454, 'output': 1.6931471805599454, 'dalam': 0.0}}
[[1.28768207 1.28768207 0.        ]
 [1.28768207 0.         1.2876

In [12]:
#UKURAN KEMIRIPAN TEKS

#edit distance

def edit_distance(string1, string2):
    if len(string1) > len(string2):
        difference = len(string1) - len(string2)
        string1[:difference]
        n  = len(string2)
    elif len(string2) > len(string1):
        difference = len(string2) - len(string1)
        string2[:difference]
        n =  len(string1)
    for i in range(n):
        if string1[i] != string2[i]:
            difference += 1
    return difference


In [13]:
print(edit_distance(doc_dict['doc1'], doc_dict['doc2']))
print(edit_distance(doc_dict['doc1'], doc_dict['doc3']))

30
31


In [14]:
#JACCARD SIMILARITY

def jaccard_sim(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1)+len(list2)) - intersection
    return float(intersection) / union


In [15]:
print(jaccard_sim(doc_dict['doc1'].split(" "),doc_dict['doc2'].split(" ")))
print(jaccard_sim(doc_dict['doc1'].split(" "),doc_dict['doc3'].split(" ")))

0.125
0.14285714285714285


In [16]:
#EUCLIDIAN DISTANCE
def euclidian_dist(vec1,vec2):
    # subtracting vector
    
    temp = vec1 - vec2
    
    #doing dot product
    #for finding
    #sum of te squares
    sum_sq = np.dot(temp.T,temp)
    
    #Doing squarefoot and
    #printing Euclidean distance
    return np.sqrt(sum_sq)



In [17]:
print(euclidian_dist(TD[:,0], TD[:,1])) #doc1 & doc2
print(euclidian_dist(TD[:,0], TD[:,2])) #doc1 & doc3

4.201188773980275
3.844897884155026


In [18]:
#COSINE SIMILARITY

import math
def cosine_sim(vec1, vec2):
    vec1 = list(vec1)
    vec2 = list(vec2)
    dot_prod = 0
    for i, v in enumerate (vec1):
        dot_prod += v * vec2[i]
    mag_1 = math.sqrt(sum([x**2 for x in vec1]))
    mag_2 = math.sqrt(sum([x**2 for x in vec2]))
    
    return dot_prod / (mag_1*mag_2)


In [19]:
print(cosine_sim(TD[:,0], TD[:,1])) #doc1 & doc2
print(cosine_sim(TD[:,0], TD[:,2])) #doc1 & doc3


0.15967058203849993
0.1832234081332565


## PENUGASAN

In [3]:
path = "C:/Users/asus/bahan python/berita"
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stemmer_text = StemmerFactory().create_stemmer()

import spacy
nlp_task = Indonesian()
corpus = []
corpus_term = []
for file in os.listdir(path): 
    with open(os.path.join(path, file), 'r') as f:
        clean_txt = re.sub("http\S+", ' ', f.read()) #menggantikan bentuk link ("https") menjadi ""
        clean_txt = re.sub("[\n\n]", ' ', clean_txt) #menggantikan double new line dengan ' '
        clean_txt = re.sub(r'[^a-zA-Z0-9 .-]',' ',clean_txt) #menggantikan semua karakter kecuali hypen(-) dan titik (.) ke ' '
        clean_txt = re.sub(r"\s+", ' ', clean_txt) #menggatikan white space berlebih dengan ' '
        clean_txt = stemmer_text.stem(clean_txt)
        corpus.append(clean_txt.lower()) #menambahkan fungsi clean_text (yang telah dilakukan perlakuan) ke corpus

print(corpus)

['wilayah kamu sudah bebas covid-19 cek 34 kab kota zona hijau baru jakarta - perintah rencana bakal terap laku batas giat masyarakat ppkm level 3 hitung 24 desember 2021 hingga 2 januari 2021 namun menteri sehat ri pasti bijak ppkm level 3 ini masih dalam tahap kaji turut direktur cegah dan kendali sakit tular langsung p2pml kemenkes ri dr siti nadia tarmizi ppkm level 3 bakal terap jika kasus covid-19 naik signifikan hal ini umum picu dengan tingkat mobilitas dan longgar protokol sehat', 'vaksin covid-19 bakal rutin tiap tahun gantung ini jelas jakarta - beri booster atau dosis tiga vaksin covid-19 di indonesia rencana bakal langsung januari 2022 lantas ada mungkin vaksinasi covid-19 bakal harus laku tiap tahun seperti vaksinasi influenza ketua satgas covid-19 ikat dokter indonesia idi prof zubairi djoerban jelas hingga kini belum ada pasti kait hal sebut turut masih ada mungkin vaksin covid-19 harus beri tiap tahun ada juga mungkin cukup booster beri sekali kemudian vaksinasi covid-

In [37]:
# 1. Tokenization
#melakukan replacement text menjadi sepenggal-sepenggal (string dan char)

corpus_term = [] #menyediakan list baru sebagai tempat hasil tokenisasi
nlp = Indonesian() #NLP bahasa

for doc in corpus:
    spacy_id = nlp(doc)
    token_kata = [token.text for token in nlp_task(doc)]
    corpus_term.append(token_kata) #menambahkan fungsi token_kata ke dalam corpus_term
    
print(corpus_term)

[['wilayah', 'kamu', 'sudah', 'bebas', 'covid', '-', '19', 'cek', '34', 'kab', 'kota', 'zona', 'hijau', 'baru', 'jakarta', '-', 'perintah', 'rencana', 'bakal', 'terap', 'laku', 'batas', 'giat', 'masyarakat', 'ppkm', 'level', '3', 'hitung', '24', 'desember', '2021', 'hingga', '2', 'januari', '2021', 'namun', 'menteri', 'sehat', 'ri', 'pasti', 'bijak', 'ppkm', 'level', '3', 'ini', 'masih', 'dalam', 'tahap', 'kaji', 'turut', 'direktur', 'cegah', 'dan', 'kendali', 'sakit', 'tular', 'langsung', 'p2pml', 'kemenkes', 'ri', 'dr', 'siti', 'nadia', 'tarmizi', 'ppkm', 'level', '3', 'bakal', 'terap', 'jika', 'kasus', 'covid', '-', '19', 'naik', 'signifikan', 'hal', 'ini', 'umum', 'picu', 'dengan', 'tingkat', 'mobilitas', 'dan', 'longgar', 'protokol', 'sehat'], ['vaksin', 'covid', '-', '19', 'bakal', 'rutin', 'tiap', 'tahun', 'gantung', 'ini', 'jelas', 'jakarta', '-', 'beri', 'booster', 'atau', 'dosis', 'tiga', 'vaksin', 'covid', '-', '19', 'di', 'indonesia', 'rencana', 'bakal', 'langsung', 'januar

In [38]:
#STOPWORDS ELIMINATION

nlp_task = Indonesian()
stop_words = nlp_task.Defaults.stop_words
corpus_term_stopwords = []

for doc in corpus:
    nlp_doc = nlp_task(doc)
    token_kata = [token.text for token in nlp_doc]
    token_stopwords = [w for w in token_kata if w not in stop_words] #melakukan pengecualian terhadap stop words
    corpus_term_stopwords.append(token_stopwords)

print(corpus_term_stopwords)

[['wilayah', 'bebas', 'covid', '-', '19', 'cek', '34', 'kab', 'kota', 'zona', 'hijau', 'jakarta', '-', 'perintah', 'rencana', 'terap', 'laku', 'batas', 'giat', 'masyarakat', 'ppkm', 'level', '3', 'hitung', '24', 'desember', '2021', '2', 'januari', '2021', 'menteri', 'sehat', 'ri', 'bijak', 'ppkm', 'level', '3', 'tahap', 'kaji', 'direktur', 'cegah', 'kendali', 'sakit', 'tular', 'langsung', 'p2pml', 'kemenkes', 'ri', 'dr', 'siti', 'nadia', 'tarmizi', 'ppkm', 'level', '3', 'terap', 'covid', '-', '19', 'signifikan', 'picu', 'tingkat', 'mobilitas', 'longgar', 'protokol', 'sehat'], ['vaksin', 'covid', '-', '19', 'rutin', 'gantung', 'jakarta', '-', 'booster', 'dosis', 'vaksin', 'covid', '-', '19', 'indonesia', 'rencana', 'langsung', 'januari', '2022', 'lantas', 'vaksinasi', 'covid', '-', '19', 'laku', 'vaksinasi', 'influenza', 'ketua', 'satgas', 'covid', '-', '19', 'ikat', 'dokter', 'indonesia', 'idi', 'prof', 'zubairi', 'djoerban', 'kait', 'vaksin', 'covid', '-', '19', 'booster', 'vaksinasi'

In [43]:
#TERM WEIGHTING

def termFrequencyInDoc(vocab, doc_dict):
    tf_docs = {}
    for doc_id in doc_dict.keys():
        tf_docs[doc_id] = {}
    for word in vocab:
        for doc_id,doc in doc_dict.items():
            tf_docs[doc_id][word] = doc.count(word)
    return tf_docs


In [44]:
def stemming_index(corpus) :
    # Stemming
    # import StemmerFactory class
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

    # create stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # stemming process
    output   = stemmer.stem(corpus)
    #for i in corpus :
    #    for word in i :
    #        output.append(stemmer.stem(word))
        
    return output


inverted_index = {}
 
for i in range(len(corpus_term_stopwords)):
    for item in corpus_term_stopwords[i]: 
        item = stemming_index(item)
        if item not in inverted_index:
            inverted_index[item] = []
        if (item in inverted_index) and ((i+1) not in inverted_index[item]):
            inverted_index[item].append(i+1)

vocab = list(inverted_index.keys())
doc_dict = {}

#clean after stemming

doc_dict['doc1'] = "wilayah kamu sudah bebas covid-19 cek 34 kab kota zona hijau baru jakarta - perintah rencana bakal terap laku batas giat masyarakat ppkm level 3 hitung 24 desember 2021 hingga 2 januari 2021 namun menteri sehat ri pasti bijak ppkm level 3 ini masih dalam tahap kaji turut direktur cegah dan kendali sakit tular langsung p2pml kemenkes ri dr siti nadia tarmizi ppkm level 3 bakal terap jika kasus covid-19 naik signifikan hal ini umum picu dengan tingkat mobilitas dan longgar protokol sehat"
doc_dict['doc2'] = "vaksin covid-19 bakal rutin tiap tahun gantung ini jelas jakarta - beri booster atau dosis tiga vaksin covid-19 di indonesia rencana bakal langsung januari 2022 lantas ada mungkin vaksinasi covid-19 bakal harus laku tiap tahun seperti vaksinasi influenza ketua satgas covid-19 ikat dokter indonesia idi prof zubairi djoerban jelas hingga kini belum ada pasti kait hal sebut turut masih ada mungkin vaksin covid-19 harus beri tiap tahun ada juga mungkin cukup booster beri sekali kemudian vaksinasi covid-19 tidak perlu lagi"
doc_dict['doc3'] = "ri mulai suntik booster di 2022 masih ampuh lawan varian delta cs jakarta - pakar aku vaksin yang sudah guna untuk dosis 1-2 memang alami turun efektivitas hadap varian baru corona seperti varian delta ingat booster atau dosis tiga vaksin covid-19 di indonesia sebut bakal mulai awal 2022 apakah jenis vaksin yang guna bakal ikut strain virus baru jawab tanya sebut ketua satgas covid-19 ikat dokter indonesia idi prof zubairi djoerban kembali singgung riset yang sudah langsung kait efektivitas vaksin covid-19 dosis 1 dan 2 ia sebut dasar riset jauh ini efektivitas vaksin covid-19 pfizer dan moderna bukti turun dalam lawan varian delta"
doc_dict['doc4'] = "alert kasus varian delta covid-19 di dki tingkat jakarta - data baru dari balitbangkes kemenkes ri per 13 november tunjuk ada tambah varian delta tambah sebut jadi di jawa barat ada 165 kasus dki jakarta 90 kasus dan sulawesi utara 86 kasus dalam satu bulan akhir balitbangkes sebut dki jakarta alami tingkat kasus varian delta yang signifikan sementara pada varian baru seperti varian alpha varian delta hingga beta banyak di indonesia asal dari dki jakarta dengan total 1 327 kasus"
doc_dict['doc5'] = "corona di as dadak naik lagi usai serang delta sempat reda jakarta - kasus covid-19 kembali naik di jumlah wilayah amerika serikat as padahal tahu covid-19 sempat catat stabil pasca serang varian delta musim panas ini ada apa hal sebut sampai oleh kepala nasihat medis gedung putih dr anthony fauci senin 15 11 2021 tahu kasus nasional turun 57 persen minggu lalu dari puncak gelombang varian delta pada musim panas namun jumlah pasien covid-19 di area barat tengah dan timur laut kini naik dadak"


print(termFrequencyInDoc(vocab, doc_dict))



{'doc1': {'wilayah': 1, 'bebas': 1, 'covid': 2, '-': 3, '19': 2, 'cek': 1, '34': 1, 'kab': 1, 'kota': 1, 'zona': 1, 'hijau': 1, 'jakarta': 1, 'perintah': 1, 'rencana': 1, 'terap': 2, 'laku': 1, 'batas': 1, 'giat': 1, 'masyarakat': 1, 'ppkm': 3, 'level': 3, '3': 4, 'hitung': 1, '24': 1, 'desember': 1, '2021': 2, '2': 7, 'januari': 1, 'menteri': 1, 'sehat': 2, 'ri': 5, 'bijak': 1, 'tahap': 1, 'kaji': 1, 'direktur': 1, 'cegah': 1, 'kendali': 1, 'sakit': 1, 'tular': 1, 'langsung': 1, 'p2pml': 1, 'kemenkes': 1, 'dr': 1, 'siti': 1, 'nadia': 1, 'tarmizi': 1, 'signifikan': 1, 'picu': 1, 'tingkat': 1, 'mobilitas': 1, 'longgar': 1, 'protokol': 1, 'vaksin': 0, 'rutin': 0, 'gantung': 0, 'booster': 0, 'dosis': 0, 'indonesia': 0, '2022': 0, 'lantas': 0, 'vaksinasi': 0, 'influenza': 0, 'ketua': 0, 'satgas': 0, 'ikat': 0, 'dokter': 0, 'idi': 0, 'prof': 0, 'zubairi': 0, 'djoerban': 0, 'kait': 0, 'suntik': 0, 'ampuh': 0, 'lawan': 0, 'varian': 0, 'delta': 0, 'cs': 0, 'pakar': 0, '1': 4, 'alami': 0, 'turu

In [45]:
def tokenisasi(text):
    tokens = text.split(" ")
    return tokens

In [46]:
def wordDocFre(vocab, doc_dict):
    df = {}
    for word in vocab:
        frq = 0
        for doc in doc_dict.values():
            if word in tokenisasi(doc):
                frq = frq + 1
        df[word] = frq
    return df
print(wordDocFre(vocab, doc_dict))

{'wilayah': 2, 'bebas': 1, 'covid': 0, '-': 5, '19': 0, 'cek': 1, '34': 1, 'kab': 1, 'kota': 1, 'zona': 1, 'hijau': 1, 'jakarta': 5, 'perintah': 1, 'rencana': 2, 'terap': 1, 'laku': 2, 'batas': 1, 'giat': 1, 'masyarakat': 1, 'ppkm': 1, 'level': 1, '3': 1, 'hitung': 1, '24': 1, 'desember': 1, '2021': 2, '2': 2, 'januari': 2, 'menteri': 1, 'sehat': 1, 'ri': 3, 'bijak': 1, 'tahap': 1, 'kaji': 1, 'direktur': 1, 'cegah': 1, 'kendali': 1, 'sakit': 1, 'tular': 1, 'langsung': 3, 'p2pml': 1, 'kemenkes': 2, 'dr': 2, 'siti': 1, 'nadia': 1, 'tarmizi': 1, 'signifikan': 2, 'picu': 1, 'tingkat': 2, 'mobilitas': 1, 'longgar': 1, 'protokol': 1, 'vaksin': 2, 'rutin': 1, 'gantung': 1, 'booster': 2, 'dosis': 2, 'indonesia': 3, '2022': 2, 'lantas': 1, 'vaksinasi': 1, 'influenza': 1, 'ketua': 2, 'satgas': 2, 'ikat': 2, 'dokter': 2, 'idi': 2, 'prof': 2, 'zubairi': 2, 'djoerban': 2, 'kait': 2, 'suntik': 1, 'ampuh': 1, 'lawan': 1, 'varian': 3, 'delta': 3, 'cs': 1, 'pakar': 1, '1': 2, 'alami': 2, 'turun': 2, 'e

In [50]:
import numpy as np

def inverseDocFre(vocab, doc_fre,length):
    idf = {}
    for word in vocab:
        idf[word] = idf[word] = 1+ np.log((length + 1)/(doc_fre[word]+1))
    return idf


In [51]:
print(inverseDocFre(vocab,wordDocFre(vocab, doc_dict),len(doc_dict)))

{'wilayah': 1.6931471805599454, 'bebas': 2.09861228866811, 'covid': 2.791759469228055, '-': 1.0, '19': 2.791759469228055, 'cek': 2.09861228866811, '34': 2.09861228866811, 'kab': 2.09861228866811, 'kota': 2.09861228866811, 'zona': 2.09861228866811, 'hijau': 2.09861228866811, 'jakarta': 1.0, 'perintah': 2.09861228866811, 'rencana': 1.6931471805599454, 'terap': 2.09861228866811, 'laku': 1.6931471805599454, 'batas': 2.09861228866811, 'giat': 2.09861228866811, 'masyarakat': 2.09861228866811, 'ppkm': 2.09861228866811, 'level': 2.09861228866811, '3': 2.09861228866811, 'hitung': 2.09861228866811, '24': 2.09861228866811, 'desember': 2.09861228866811, '2021': 1.6931471805599454, '2': 1.6931471805599454, 'januari': 1.6931471805599454, 'menteri': 2.09861228866811, 'sehat': 2.09861228866811, 'ri': 1.4054651081081644, 'bijak': 2.09861228866811, 'tahap': 2.09861228866811, 'kaji': 2.09861228866811, 'direktur': 2.09861228866811, 'cegah': 2.09861228866811, 'kendali': 2.09861228866811, 'sakit': 2.0986122

In [52]:
#VECTOR SPACE MODEL

def tfidf(vocab, tf, idf_scr, doc_dict):
    tf_idf_scr = {}
    for doc_id in doc_dict.keys():
        tf_idf_scr[doc_id] = {}
    for word in vocab:
        for doc_id,doc in doc_dict.items():
            tf_idf_scr[doc_id][word] = tf[doc_id][word] * idf_scr[word]
    return tf_idf_scr

In [53]:
tf_idf = tfidf(vocab, termFrequencyInDoc(vocab, doc_dict), inverseDocFre(vocab, wordDocFre(vocab, doc_dict),len(doc_dict)), doc_dict)
print(tf_idf)

# Term - Document, Matrix

TD = np.zeros((len(vocab),len(doc_dict)))
for word in vocab: 
    for doc_id, doc in tf_idf.items():
        ind1 = vocab.index(word)
        ind2 = list(tf_idf.keys()).index(doc_id)
        TD[ind1][ind2] = tf_idf[doc_id][word]
print(TD)

{'doc1': {'wilayah': 1.6931471805599454, 'bebas': 2.09861228866811, 'covid': 5.58351893845611, '-': 3.0, '19': 5.58351893845611, 'cek': 2.09861228866811, '34': 2.09861228866811, 'kab': 2.09861228866811, 'kota': 2.09861228866811, 'zona': 2.09861228866811, 'hijau': 2.09861228866811, 'jakarta': 1.0, 'perintah': 2.09861228866811, 'rencana': 1.6931471805599454, 'terap': 4.19722457733622, 'laku': 1.6931471805599454, 'batas': 2.09861228866811, 'giat': 2.09861228866811, 'masyarakat': 2.09861228866811, 'ppkm': 6.29583686600433, 'level': 6.29583686600433, '3': 8.39444915467244, 'hitung': 2.09861228866811, '24': 2.09861228866811, 'desember': 2.09861228866811, '2021': 3.386294361119891, '2': 11.852030263919618, 'januari': 1.6931471805599454, 'menteri': 2.09861228866811, 'sehat': 4.19722457733622, 'ri': 7.027325540540822, 'bijak': 2.09861228866811, 'tahap': 2.09861228866811, 'kaji': 2.09861228866811, 'direktur': 2.09861228866811, 'cegah': 2.09861228866811, 'kendali': 2.09861228866811, 'sakit': 2.09

In [54]:
#UKURAN KEMIRIPAN TEKS

#edit distance

def edit_distance(string1, string2):
    if len(string1) > len(string2):
        difference = len(string1) - len(string2)
        string1[:difference]
        n  = len(string2)
    elif len(string2) > len(string1):
        difference = len(string2) - len(string1)
        string2[:difference]
        n =  len(string1)
    for i in range(n):
        if string1[i] != string2[i]:
            difference += 1
    return difference

In [65]:
print("doc1 ke doc lain")
print(edit_distance(doc_dict['doc1'], doc_dict['doc2']))
print(edit_distance(doc_dict['doc1'], doc_dict['doc3']))
print(edit_distance(doc_dict['doc1'], doc_dict['doc4']))
print(edit_distance(doc_dict['doc1'], doc_dict['doc5']))

print("\ndoc2 ke doc lain")
print(edit_distance(doc_dict['doc2'], doc_dict['doc3']))
print(edit_distance(doc_dict['doc2'], doc_dict['doc4']))
print(edit_distance(doc_dict['doc2'], doc_dict['doc5']))

print("\ndoc3 ke doc lain")
print(edit_distance(doc_dict['doc3'], doc_dict['doc4']))
print(edit_distance(doc_dict['doc3'], doc_dict['doc5']))

print("\ndoc4 ke doc lain")
print(edit_distance(doc_dict['doc4'], doc_dict['doc5']))


doc1 ke doc lain
493
600
442
459

doc2 ke doc lain
592
491
484

doc3 ke doc lain
601
587

doc4 ke doc lain
460


In [66]:
#JACCARD SIMILARITY

def jaccard_sim(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1)+len(list2)) - intersection
    return float(intersection) / union

In [74]:
print("doc1 ke doc lain")
print(jaccard_sim(doc_dict['doc1'].split(" "),doc_dict['doc2'].split(" ")))
print(jaccard_sim(doc_dict['doc1'].split(" "),doc_dict['doc3'].split(" ")))
print(jaccard_sim(doc_dict['doc1'].split(" "),doc_dict['doc4'].split(" ")))
print(jaccard_sim(doc_dict['doc1'].split(" "),doc_dict['doc5'].split(" ")))

print("\ndoc2 ke doc lain")
print(jaccard_sim(doc_dict['doc2'].split(" "),doc_dict['doc3'].split(" ")))
print(jaccard_sim(doc_dict['doc2'].split(" "),doc_dict['doc4'].split(" ")))
print(jaccard_sim(doc_dict['doc2'].split(" "),doc_dict['doc5'].split(" ")))

print("\ndoc3 ke doc lain")
print(jaccard_sim(doc_dict['doc3'].split(" "),doc_dict['doc4'].split(" ")))
print(jaccard_sim(doc_dict['doc3'].split(" "),doc_dict['doc5'].split(" ")))

print("\ndoc4 ke doc lain")
print(jaccard_sim(doc_dict['doc4'].split(" "),doc_dict['doc5'].split(" ")))

doc1 ke doc lain
0.09395973154362416
0.07514450867052024
0.08666666666666667
0.07741935483870968

doc2 ke doc lain
0.16560509554140126
0.059602649006622516
0.06493506493506493

doc3 ke doc lain
0.09580838323353294
0.06857142857142857

doc4 ke doc lain
0.08609271523178808


In [75]:
#EUCLIDIAN DISTANCE
def euclidian_dist(vec1,vec2):
    # subtracting vector
    
    temp = vec1 - vec2
    
    #doing dot product
    #for finding
    #sum of te squares
    sum_sq = np.dot(temp.T,temp)
    
    #Doing squarefoot and
    #printing Euclidean distance
    return np.sqrt(sum_sq)


In [76]:
print("doc1 ke doc lain")
print(euclidian_dist(TD[:,0], TD[:,1])) #doc1 & doc2
print(euclidian_dist(TD[:,0], TD[:,2])) #doc1 & doc3
print(euclidian_dist(TD[:,0], TD[:,3])) #doc1 & doc4
print(euclidian_dist(TD[:,0], TD[:,4])) #doc1 & doc5

print("\ndoc2 ke doc lain")
print(euclidian_dist(TD[:,1], TD[:,2])) #doc2 & doc3
print(euclidian_dist(TD[:,1], TD[:,3])) #doc2 & doc4
print(euclidian_dist(TD[:,1], TD[:,4])) #doc2 & doc5

print("\ndoc3 ke doc lain")
print(euclidian_dist(TD[:,2], TD[:,3])) #doc3 & doc4
print(euclidian_dist(TD[:,2], TD[:,4])) #doc3 & doc5

print("\ndoc4 ke doc lain")
print(euclidian_dist(TD[:,3], TD[:,4])) #doc4 & doc5


doc1 ke doc lain
29.983699021395978
28.65294953743506
26.8302339516762
26.82095529351533

doc2 ke doc lain
21.87701427936018
31.69970282920237
24.62421295976927

doc3 ke doc lain
28.515254343453687
26.715590424016575

doc4 ke doc lain
24.50345477227729


In [77]:
#COSINE SIMILARITY

import math
def cosine_sim(vec1, vec2):
    vec1 = list(vec1)
    vec2 = list(vec2)
    dot_prod = 0
    for i, v in enumerate (vec1):
        dot_prod += v * vec2[i]
    mag_1 = math.sqrt(sum([x**2 for x in vec1]))
    mag_2 = math.sqrt(sum([x**2 for x in vec2]))
    
    return dot_prod / (mag_1*mag_2)


In [80]:
print("doc1 ke doc lain")
print(cosine_sim(TD[:,0], TD[:,1])) #doc1 & doc2
print(cosine_sim(TD[:,0], TD[:,2])) #doc1 & doc3
print(cosine_sim(TD[:,0], TD[:,3])) #doc1 & doc4
print(cosine_sim(TD[:,0], TD[:,4])) #doc1 & doc5

print("\ndoc2 ke doc lain")
print(cosine_sim(TD[:,1], TD[:,2])) #doc2 & doc3
print(cosine_sim(TD[:,1], TD[:,3])) #doc2 & doc4
print(cosine_sim(TD[:,1], TD[:,4])) #doc2 & doc5

print("\ndoc3 ke doc lain")
print(cosine_sim(TD[:,2], TD[:,3])) #doc3 & doc4
print(cosine_sim(TD[:,2], TD[:,4])) #doc3 & doc5

print("\ndoc4 ke doc lain")
print(cosine_sim(TD[:,3], TD[:,4])) #doc4 & doc5

doc1 ke doc lain
0.6149771001307007
0.6167955480053066
0.561905856717853
0.6255448277386594

doc2 ke doc lain
0.8162799188325175
0.5501839798971456
0.7546956103451156

doc3 ke doc lain
0.6045719160702497
0.6847538653474112

doc4 ke doc lain
0.6716031905461545
